In [2]:
pip install git+https://github.com/alberanid/imdbpy

  Cloning https://github.com/alberanid/imdbpy to /tmp/pip-req-build-ojfvtaru
  Running command git clone -q https://github.com/alberanid/imdbpy /tmp/pip-req-build-ojfvtaru
  Created wheel for IMDbPY: filename=IMDbPY-2021.10.24-py3-none-any.whl size=300551 sha256=9a25eaa42d34ef7d5ef5361c310ced948b74166286c9118f33c521d05108423c
  Stored in directory: /tmp/pip-ephem-wheel-cache-2zcc4bsm/wheels/d1/75/06/4d048b85021fb5f979ece79f70f27550ed04b76f9603dcb0d9
Successfully built IMDbPY


In [28]:
import json
import spacy
import re
import nltk
import difflib
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
from nltk.corpus import stopwords
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
from imdb import IMDb
import math
from collections import Counter
WORD = re.compile(r'\w+')
ia = IMDb()

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [91]:
mystop_words = stopwords.words('english')
mystop_words.extend(['performance','made','-','congrats','congratulations','got','knows','well','well-'])
print(mystop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
PEOPLE_AWARDS = ['best director - motion picture','best performance by an actress in a television series - comedy or musical','best performance by an actor in a supporting role in a motion picture','best performance by an actress in a supporting role in a series, mini-series or motion picture made for television','best performance by an actress in a motion picture - comedy or musical','best performance by an actress in a television series - drama','best performance by an actress in a motion picture - drama',	'best performance by an actor in a motion picture - comedy or musical','best performance by an actor in a supporting role in a series, mini-series or motion picture made for television','best performance by an actress in a supporting role in a motion picture','best performance by an actor in a mini-series or motion picture made for television','best performance by an actress in a mini-series or motion picture made for television','best performance by an actor in a motion picture - drama', 'best performance by an actor in a television series - drama','best performance by an actor in a television series - comedy or musical']
people_keywords = 'actor|actress|supporting|role|performance|director'
MOVIE_AWARDS = ['best screenplay - motion picture','best television series - comedy or musical','best original song - motion picture','best motion picture - comedy or musical','best foreign language film','best mini-series or motion picture made for television','best motion picture - drama','best television series - drama','best animated feature film','best original score - motion picture',]

In [33]:
with open('sample_data/movies_2006_onwards.txt') as f:
  movie_names = f.readlines()
for i in range(len(movie_names)):
  movie_names[i] = movie_names[i][:-1].lower()

In [6]:
def open_jsonfile(data_path = 'sample_data/gg2013.json'):
  file_ = open(data_path)
  data = json.load(file_)
  return data

In [7]:
def clean_award_name(award_name):
  #removes stopwords from award name and returns key words in award name as list
  award_name = award_name.lower()
  cleaned_award = []
  split = award_name.split()
  for word in split:
    if word not in mystop_words:  # Many words are irrelevant, removed with list of stopwords
      cleaned_award.append(word)
      if word == 'television':  # Usually television is shortened to tv, want to account for both
        cleaned_award.append('tv')
  return cleaned_award

#print(clean_award_name("best performance by an actor in a motion picture - drama"))

In [39]:
def award_relevant_tweets(tweet_data, award_keywords):
  relevant_tweets = []
  for tweet in tweet_data:
    text = tweet['text']
    text = text.lower()
    check = 0
    for key_word in award_keywords:
      if key_word not in text:
        check +=1
    if re.search(people_keywords,text):
      continue
    if 'tv' in award_keywords:
      if check <2:
        relevant_tweets.append(tweet['text'])
    else:
      if check ==0:
        relevant_tweets.append(tweet['text'])
    """else:
      if winner.lower() in text:
        relevant_tweets.append(tweet['text'])"""
  return relevant_tweets

######testing function###########
#data = open_jsonfile(data_path = 'sample_data/gg2013.json')
#award_keywords = clean_award_name('best performance by an actress in a motion picture - drama')
#rel_tweets = award_relevant_tweets(data,award_keywords,winner = 'Jessica Chastain')
#for rel_tweet in rel_tweets:
#  print(rel_tweet)


In [24]:
def remove_stop_words(small_tweet,mystop_words = mystop_words):
  small_tweet = small_tweet.lower()
  good_tweet = []
  split = small_tweet.split()
  for word in split:
    if re.search('http',word.lower()):
      continue
    if word.lower() not in mystop_words:
      good_tweet.append(word)
  return(' '.join(good_tweet))

###########testing function##########
#small_tweet = 'Golden Globes: Winner for Best Supporting Actor in a Motion Picture is Christoph Waltz for "Django Unchained"'
#print(remove_stop_words('small_tweet here is the link https://asfasdfsd.com'))

In [74]:
def add_to_dictionary(potential_nominees, small_tweet, weightage):
  small_tweet = re.sub('#([a-zA-Z0-9_]{1,50})', '', small_tweet)
  small_tweet = re.sub('@([a-zA-Z0-9_]{1,50})', '', small_tweet)
  small_tweet = re.sub(r'[^\w\s]','',small_tweet)
  if small_tweet in potential_nominees.keys():
    potential_nominees[small_tweet] += weightage
  else:
    potential_nominees[small_tweet] = weightage

In [72]:
small_tweet = 'Congrats \'Game Change\' for #GoldenGlobes2013 #2013gg winning. Best#goldenglobes Mini-Series? or #golden Motion P"icture Made, for Television!'
small_tweet = re.sub('#([a-zA-Z0-9_]{1,50})', '', small_tweet)
print(s)

Congrats Game Change for winning Best MiniSeries or Motion Picture Made for Television


In [20]:
def populate_potential_movie_nominees(filtered_tweets,award_keywords):
  stretch = 3
  potential_nominees = {}
  prior_keywords = ['won','win','deserve','deserving','deserved','nomination','nominated','snubbed','better']
  posterior_keywords = ['nominee','deserved award for','preferred','winner','goes to','over']
  biside_keywords = ['beat','rob']
  negative_indicating_keywords = 'nominated|nominee'
  negative_keywords = 'should have|did not|should\'ve|not'

  for tweet in filtered_tweets:
    if 'rt @' in tweet.lower():
      continue
    if re.search(negative_indicating_keywords,tweet) and re.search(negative_keywords,tweet):
      continue
    for key_word in prior_keywords:
      if key_word in tweet:
        small_tweet = tweet[:re.search(key_word,tweet).start()]
        small_tweet = remove_stop_words(small_tweet)
        tokens = small_tweet.split()
        if len(tokens) == 0:
          continue
        elif len(tokens) <= stretch:
          small_tweet = ' '.join(tokens)
          #small_tweet = remove_stop_words(small_tweet)
          add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
        else:
          small_tweet = ' '.join(tokens[-stretch::1])
          #small_tweet = remove_stop_words(small_tweet)
          add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
    for key_word in posterior_keywords:
      if key_word in tweet:
        small_tweet = tweet[re.search(key_word,tweet).start():]
        small_tweet = remove_stop_words(small_tweet)
        tokens = small_tweet.split(' ')
        if len(tokens) <= 1:
          continue
        tokens = tokens[1::]
        if len(tokens) <= stretch:
          small_tweet = ' '.join(tokens)
          #small_tweet = remove_stop_words(small_tweet)
          add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
        else:
          small_tweet = ' '.join(tokens[-stretch::1])
          #small_tweet = remove_stop_words(small_tweet)
          add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
    for key_word in biside_keywords:
      if key_word in tweet:
        small_tweet_left = tweet[:re.search(key_word,tweet).start()]
        small_tweet_right = tweet[re.search(key_word,tweet).start():]
        small_tweet_left = remove_stop_words(small_tweet_left)
        small_tweet_right = remove_stop_words(small_tweet_right)
        tokens_left = small_tweet_left.split(' ')
        tokens_right = small_tweet_right.split(' ')
        if len(tokens_left) > 0:
          if len(tokens_left) <= stretch:
            small_tweet = ' '.join(tokens_left)
            #small_tweet = remove_stop_words(small_tweet)
            add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
          else:
            small_tweet = ' '.join(tokens_left[-stretch::1])
            #small_tweet = remove_stop_words(small_tweet)
            add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
        if len(tokens_right) > 1:
          tokens_right = tokens_right[1::]
          if len(tokens_right) <= stretch:
            small_tweet = ' '.join(tokens_right)
            #small_tweet = remove_stop_words(small_tweet)
            add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
          else:
            small_tweet = ' '.join(tokens[-stretch::1])
            #small_tweet = remove_stop_words(small_tweet)
            add_to_dictionary(potential_nominees,small_tweet,weightage = 20)
  return potential_nominees
###################testing function#####################
#tweet_data = open_jsonfile(data_path = 'sample_data/gg2013.json')
#award_keywords = clean_award_name("best supporting actress in a motion picture")
#relevant_tweets = award_relevant_tweets(tweet_data, award_keywords)
#potential_nominees = populate_potential_movie_nominees(relevant_tweets)
#for key in potential_nominees.keys():
#  print(key,potential_nominees[key])

In [12]:
def get_cosine(vec1, vec2):
    # print vec1, vec2
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    return Counter(WORD.findall(text))

def cosine_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())

    return get_cosine(a, b)

##################testing function###############
#print(cosine_similarity('steven ','steven speilberg'))

In [13]:
def char_similarity(a,b):
  final_similarity = 0
  a = a.lower()
  b = b.lower()
  a_tokens = a.split()
  b_tokens = b.split()
  for a_token in a_tokens:
    for b_token in b_tokens:
      similarity = 0
      if len(a_token) > len(b_token):
        bigger_token = a_token
        smaller_token = b_token
      else:
        bigger_token = b_token
        smaller_token = a_token
      for i in range(len(smaller_token)):
        if smaller_token[i] == bigger_token[i]:
          similarity += 1
      if final_similarity < similarity/len(smaller_token):
        final_similarity = similarity/len(smaller_token)
  return final_similarity

##############testing function###################
print(char_similarity('ang lee','anger'))

1.0


In [14]:
def merge_potential_nominees(potential_nominees):
  compressed_nominees = {}
  potential_nominees = sorted(potential_nominees.items(), key = lambda k: -k[1])
  potential_nominees = dict(potential_nominees)
  for key1 in potential_nominees.keys():
    if len(compressed_nominees) == 0:
      compressed_nominees[key1] = potential_nominees[key1]
    else:
      for key2 in list(compressed_nominees.keys()):
        if cosine_similarity(key1,key2) >= 0.6:
          compressed_nominees[key2] += potential_nominees[key1]
          break
        else:
          if key2 == list(compressed_nominees.keys())[-1]:
            compressed_nominees[key1] = potential_nominees[key1]
  return compressed_nominees

##############testing function###################
#tweet_data = open_jsonfile(data_path = 'sample_data/gg2013.json')
#award_keywords = clean_award_name("best supporting actress in a motion picture")
#relevant_tweets = award_relevant_tweets(tweet_data, award_keywords)
#potential_nominees = populate_potential_movie_nominees(relevant_tweets)
#merged_nominees = merge_potential_nominees(potential_nominees)
#for key in merged_nominees.keys():
#  print(key,merged_nominees[key])

In [15]:
def check_on_imdb(compressed_nominees):
  result_nominees = []
  count = 0
  for key in compressed_nominees.keys():
    movies_identified = ia.search_person(key)

    if len(movies_identified) == 0:
      continue
    if cosine_similarity(movies_identified[0]['name'],key) >= 0.6:
      result_nominees.append(movies_identified[0]['name'])
      count += 1
    if count == 5 :
      break
  return result_nominees
################testing function####################
#result_nominees = check_on_imdb(merged_nominees)
#print(result_nominees)

In [ ]:
def remove_hashtag_punctuation(nominees_list):
  cleaned_nominees = {}
  for nominee in nominee_list:


In [90]:
def check_on_movie_list(movie_names,merged_potential_nominees):
  result_list = []
  count = 0
  for key in merged_potential_nominees.keys():
    movies = difflib.get_close_matches(key,movie_names,cutoff = 0.75)
    print(key,movies)
    if len(movies) != 0:
      result_list.append(movies[0])
      count+= 1
    if count == 5:
      break
  return result_list
print(difflib.get_close_matches('skyfall skyfall',movie_names,cutoff = 0.64))

[]


In [35]:
def find_nominees(award_name):
  #open the json file of tweets
  tweet_data = open_jsonfile()
  #clean the award name and take it as list of tokens
  award_keywords = clean_award_name(award_name)
  #find all tweets relevant to this award name
  filtered_tweets = award_relevant_tweets(tweet_data,award_keywords)
  #for tweet in filtered_tweets:
    #print(tweet)
  #print(filtered_tweets)
  #find potential nominees from relevant tweets
  potential_nominees = populate_potential_movie_nominees(filtered_tweets)
  #print(potential_nominees)
  #merge potential nominees if there are various version of same nominees
  merged_potential_nominees = merge_potential_nominees(potential_nominees)
  print(merged_potential_nominees)
  #query imdb for best five results
  #results = check_on_imdb(merged_potential_nominees)
  results = check_on_movie_list(movie_names, merged_potential_nominees)
  #return top five results
  return results
#winners = ['Ben Affleck','Lena Dunham']


In [84]:
for i in range(len(MOVIE_AWARDS)):
  print('-----------------------------')
  print(MOVIE_AWARDS[i])
  print(find_nominees(MOVIE_AWARDS[i]))
  print('-----------------------------')

-----------------------------
best screenplay - motion picture
{'django unchained ': 140, 'quentin tarantino': 80, 'hell yea': 40, 'see this ': 40, 'motion picture ': 80, 'unchained yesss ': 20, ' yes ': 20, '  ': 20, '': 20}
django unchained  ['django unchained']
quentin tarantino []
hell yea []
see this  []
motion picture  []
unchained yesss  []
 yes  []
   []
 []
['django unchained']
-----------------------------
-----------------------------
best television series - comedy or musical
{'girls': 180, '': 60, 'series comedy musical': 40, ' sucks': 20, 'to ': 20, 'hazzah  ': 20, 'wooo hooo girls': 20, 'again cause ': 20, 'family  ': 20}
girls ['girl', 'irl']
 []
series comedy musical []
 sucks ['suck']
to  ['to']
hazzah   []
wooo hooo girls []
again cause  []
family   ['family', 'family', 'framily']
['girl', 'suck', 'to', 'family']
-----------------------------
-----------------------------
best original song - motion picture
{'motion picture song': 40, ' skyfall skyfall': 140, '': 40,

In [17]:
find_nominees('best television series - drama')

Game Change wins Best TV Mini Series/Drama #GoldenGlobes
RT @MTVNews: Congrats 'Game Change' for winning Best Mini-Series or Motion Picture Made for Television! http://t.co/Z36SYljF #GoldenGlobes
Congrats 'Game Change' for winning Best Mini-Series or Motion Picture Made for Television! http://t.co/x4utVC0i #GoldenGlobes @MTVNews
RT @MTVNews: Congrats 'Game Change' for winning Best Mini-Series or Motion Picture Made for Television! http://t.co/Z36SYljF #GoldenGlobes
RT @MTVNews: Congrats 'Game Change' for winning Best Mini-Series or Motion Picture Made for Television! http://t.co/Z36SYljF #GoldenGlobes
RT @AshaTomlinson: Game Change wins Best TV Mini Series/Drama #GoldenGlobes
Oh und jetzt kommt direkt Best TV-Series Drama! #GoldenGlobes
Aaaaaand #Homeland takes Best Drama TV Series. #Psychic #GoldenGlobes
Best TV Series (Drama): HOMELAND #GoldenGlobes
The best TV series (DRAMA) goes to #HOMELAND!!!! #GoldenGlobes
RT @goldenglobes: Best Television Series - Drama - Homeland (@SHO_Homelan